# **Final Project**

##**Problem stament :**     

The widespread dissemination of fake news and propaganda presents serious societal risks, including the erosion of public trust, political polarization, manipulation of elections, and the spread of harmful misinformation during crises such as pandemics or conflicts. From an NLP perspective, detecting fake news is fraught with challenges. Linguistically, fake news often mimics the tone and structure of legitimate journalism, making it difficult to distinguish using surface-level features. The absence of reliable and up-to-date labeled datasets, especially across multiple languages and regions, hampers the effectiveness of supervised learning models. Additionally, the dynamic and adversarial nature of misinformation means that malicious actors constantly evolve their language and strategies to bypass detection systems. Cultural context, sarcasm, satire, and implicit bias further complicate automated analysis. Moreover, NLP models risk amplifying biases present in training data, leading to unfair classifications and potential censorship of legitimate content. These challenges underscore the need for cautious, context-aware approaches, as the failure to address them can inadvertently contribute to misinformation, rather than mitigate it.



Use datasets in link : https://drive.google.com/drive/folders/1mrX3vPKhEzxG96OCPpCeh9F8m_QKCM4z?usp=sharing
to complete requirement.

## **About dataset:**

* **True Articles**:

  * **File**: `MisinfoSuperset_TRUE.csv`
  * **Sources**:

    * Reputable media outlets like **Reuters**, **The New York Times**, **The Washington Post**, etc.

* **Fake/Misinformation/Propaganda Articles**:

  * **File**: `MisinfoSuperset_FAKE.csv`
  * **Sources**:

    * **American right-wing extremist websites** (e.g., Redflag Newsdesk, Breitbart, Truth Broadcast Network)
    * **Public dataset** from:

      * Ahmed, H., Traore, I., & Saad, S. (2017): "Detection of Online Fake News Using N-Gram Analysis and Machine Learning Techniques" *(Springer LNCS 10618)*



## **Requirement**

A team consisting of three members must complete a project that involves applying the methods learned from the beginning of the course up to the present. The team is expected to follow and document the entire machine learning workflow, which includes the following steps:

1. **Data Preprocessing**: Clean and prepare the dataset,etc.

2. **Exploratory Data Analysis (EDA)**: Explore and visualize the data.

3. **Model Building**: Select and build one or more machine learning models suitable for the problem at hand.

4. **Hyperparameter set up**: Set and adjust the model's hyperparameters using appropriate methods to improve performance.

5. **Model Training**: Train the model(s) on the training dataset.

6. **Performance Evaluation**: Evaluate the trained model(s) using appropriate metrics (e.g., accuracy, precision, recall, F1-score, confusion matrix, etc.) and validate their performance on unseen data.

7. **Conclusion**: Summarize the results, discuss the model's strengths and weaknesses, and suggest possible improvements or future work.





In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"C:\Users\Admin\OneDrive - VNU-HCMUS\cleaned")
print(df.head())
print(df.shape)

                                                text  label  text_length
0  link imo happening guys jumping conclusions co...      1         2717
1  house passed budget today slim margin democrat...      1         3203
2  youtube video posted january cloudgate studio ...      0         2549
3  marseille france sang danced chanted even reas...      0         8149
4  sunday nbc meet press discussing president don...      0         1235
(68604, 3)


In [3]:
# Kiểm tra dữ liệu rỗng
df.isnull().sum()

text           7
label          0
text_length    0
dtype: int64

In [4]:
# Loại bỏ các hàng có giá trị NaN trong cột 'text'
df = df.dropna(subset=['text'])

In [5]:
# Chia dữ liệu thành 3 phần: train, valid, test
# 60% train, 20% valid, 20% test
train_df = df[:int(0.6 * len(df))]
valid_df = df[int(0.6 * len(df)):int(0.8 * len(df))]
test_df = df[int(0.8 * len(df)):]

# Tokenize data

In [6]:
# Tokenize dữ liệu
from transformers import BertTokenizerFast
from transformers import XLNetTokenizerFast

class Tokenizer:
    def __init__(self, model_name, max_length=128):
        if 'bert' in model_name:
            self.tokenizer = BertTokenizerFast.from_pretrained(model_name)
        elif 'xlnet' in model_name:
            self.tokenizer = XLNetTokenizerFast.from_pretrained(model_name)
        self.max_length = max_length

    def tokenize_function(self, examples):
        return self.tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=self.max_length
        )

from datasets import Dataset

tokenizer = Tokenizer('bert-base-uncased')

# Convert DataFrames to HuggingFace Datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize datasets
train_dataset = train_dataset.map(tokenizer.tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenizer.tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenizer.tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
valid_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/41158 [00:00<?, ? examples/s]

Map:   0%|          | 0/13719 [00:00<?, ? examples/s]

Map:   0%|          | 0/13720 [00:00<?, ? examples/s]

# Build and Load model

In [7]:
from transformers import BertForSequenceClassification, XLNetForSequenceClassification

class ModelBuilder:
    def __init__(self, num_labels=2):
        self.num_labels = num_labels
        self.model = None

    def build_model(self, model_name):
        if 'bert' in model_name:
            self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels)
        elif 'xlnet' in model_name:
            self.model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels)
        return self.model

In [8]:
model_builder = ModelBuilder()
bert_model = model_builder.build_model('bert-base-uncased')
xlnet_model = model_builder.build_model('xlnet-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
xlnet_model.to(device)

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

# Train model

In [10]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': float(f1),
        'precision': float(precision),
        'recall': float(recall)
    }

data_collator = DataCollatorWithPadding(tokenizer=tokenizer.tokenizer)

training_args = TrainingArguments(
    output_dir='./bert_results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=2,
    load_best_model_at_end=True,
    dataloader_num_workers=8,
    fp16=True
)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Bert model

In [10]:
# Training BERT model
bert_trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer.tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
bert_trainer.train()

  0%|          | 0/5145 [00:00<?, ?it/s]

{'loss': 0.4966, 'grad_norm': 2.865341901779175, 'learning_rate': 4.517006802721089e-05, 'epoch': 0.1}
{'loss': 0.3775, 'grad_norm': 51.16004943847656, 'learning_rate': 4.0310981535471334e-05, 'epoch': 0.19}
{'loss': 0.3229, 'grad_norm': 6.9868292808532715, 'learning_rate': 3.546161321671526e-05, 'epoch': 0.29}
{'loss': 0.2944, 'grad_norm': 121.6175537109375, 'learning_rate': 3.0602526724975704e-05, 'epoch': 0.39}
{'loss': 0.2704, 'grad_norm': 0.05791429802775383, 'learning_rate': 2.5743440233236154e-05, 'epoch': 0.49}
{'loss': 0.2576, 'grad_norm': 6.471226215362549, 'learning_rate': 2.0884353741496597e-05, 'epoch': 0.58}
{'loss': 0.2283, 'grad_norm': 1.7582225799560547, 'learning_rate': 1.6044703595724002e-05, 'epoch': 0.68}
{'loss': 0.2402, 'grad_norm': 0.12386985123157501, 'learning_rate': 1.1185617103984451e-05, 'epoch': 0.78}
{'loss': 0.2174, 'grad_norm': 0.8838419318199158, 'learning_rate': 6.3265306122448975e-06, 'epoch': 0.87}
{'loss': 0.2136, 'grad_norm': 0.38857510685920715, 

  0%|          | 0/1715 [00:00<?, ?it/s]

{'eval_loss': 0.1977662593126297, 'eval_accuracy': 0.9532764778773963, 'eval_f1': 0.9533051801042925, 'eval_precision': 0.9538110833988157, 'eval_recall': 0.9532764778773963, 'eval_runtime': 125.9903, 'eval_samples_per_second': 108.889, 'eval_steps_per_second': 13.612, 'epoch': 1.0}
{'train_runtime': 1357.8556, 'train_samples_per_second': 30.311, 'train_steps_per_second': 3.789, 'train_loss': 0.29085316560706315, 'epoch': 1.0}


TrainOutput(global_step=5145, training_loss=0.29085316560706315, metrics={'train_runtime': 1357.8556, 'train_samples_per_second': 30.311, 'train_steps_per_second': 3.789, 'total_flos': 5414562408253440.0, 'train_loss': 0.29085316560706315, 'epoch': 1.0})

In [ ]:
# Evaluate the model
bert_evaluate = bert_trainer.evaluate(eval_dataset=test_dataset)
print(f"Evaluation results: {bert_evaluate}")

  0%|          | 0/1715 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 0.20656894147396088, 'eval_accuracy': 0.9505830903790088, 'eval_f1': 0.950670697794864, 'eval_precision': 0.9514131678200916, 'eval_recall': 0.9505830903790088, 'eval_runtime': 120.8432, 'eval_samples_per_second': 113.536, 'eval_steps_per_second': 14.192, 'epoch': 1.0}


## XLNet model

In [11]:
# Training XLNet model
xlnet_trainer = Trainer(
    model=xlnet_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer.tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
xlnet_trainer.train()

  0%|          | 0/2572 [00:00<?, ?it/s]

{'loss': 0.7063, 'grad_norm': 6.802664756774902, 'learning_rate': 4.0338258164852254e-05, 'epoch': 0.19}
{'loss': 0.686, 'grad_norm': 15.932848930358887, 'learning_rate': 3.0618195956454124e-05, 'epoch': 0.39}
{'loss': 0.6734, 'grad_norm': 8.500238418579102, 'learning_rate': 2.089813374805599e-05, 'epoch': 0.58}
{'loss': 0.6735, 'grad_norm': 8.59803295135498, 'learning_rate': 1.1178071539657855e-05, 'epoch': 0.78}
{'loss': 0.6667, 'grad_norm': 3.528632879257202, 'learning_rate': 1.4580093312597201e-06, 'epoch': 0.97}


  0%|          | 0/1715 [00:00<?, ?it/s]

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


{'eval_loss': 1.6011463403701782, 'eval_accuracy': 0.47270209198921204, 'eval_f1': 0.30345209528311623, 'eval_precision': 0.22344726777097748, 'eval_recall': 0.47270209198921204, 'eval_runtime': 134.4547, 'eval_samples_per_second': 102.034, 'eval_steps_per_second': 12.755, 'epoch': 1.0}
{'train_runtime': 1065.9724, 'train_samples_per_second': 38.611, 'train_steps_per_second': 2.413, 'train_loss': 0.6808119131654825, 'epoch': 1.0}


TrainOutput(global_step=2572, training_loss=0.6808119131654825, metrics={'train_runtime': 1065.9724, 'train_samples_per_second': 38.611, 'train_steps_per_second': 2.413, 'total_flos': 2930848316030976.0, 'train_loss': 0.6808119131654825, 'epoch': 0.9998056365403304})

In [12]:
# Evaluate the model
xlnet_evaluate = xlnet_trainer.evaluate(eval_dataset=test_dataset)
print(f"Evaluation results: {xlnet_evaluate}")

  0%|          | 0/1715 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 1.6875672340393066, 'eval_accuracy': 0.44329446064139943, 'eval_f1': 0.2723075355641845, 'eval_precision': 0.19650997883534924, 'eval_recall': 0.44329446064139943, 'eval_runtime': 97.8479, 'eval_samples_per_second': 140.218, 'eval_steps_per_second': 17.527, 'epoch': 0.9998056365403304}


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
